<a href="https://colab.research.google.com/github/harnalashok/general/blob/master/synthetic_tabular_data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Last amended: 2nd July, 2021
# Myfolder: lubuntu_healthcare VM
# Ref: https://sdv.dev/
#      https://github.com/sdv-dev/CTGAN
#
# Objective: Generate Synthetic data using CTGAN


We generate 1lakh synthetic samples of that from big data mart. <br>My OneDrive folder: D:\data\OneDrive\Documents\big_mart_sales_problem

## Install CTGAN Synthetic Data Generator

In [ ]:
# Refer
!pip install ctgan

## Call libraries

In [3]:
from ctgan import CTGANSynthesizer

In [4]:
import numpy as np
import pandas as pd
import os
import time

In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Read data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [6]:
path = "/content/drive/MyDrive/Colab_data_files/big_data_mart"
os.chdir(path)

In [57]:
df = pd.read_csv("bigdatamart.csv")
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


### Nulls must be filled in

In [58]:
# Check nulls in features
df.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [59]:
# Begin filling up
df['Outlet_Size'].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [60]:
df['Outlet_Size']= df['Outlet_Size'].fillna(value = 'Medium')

In [61]:
df['Item_Weight'].median()

12.6

In [62]:
df['Item_Weight'] = df['Item_Weight'].fillna(value = 12.6)

In [63]:
# Check again
df.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

## Process for data generation

In [13]:
print(list(df.columns))

['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_Type', 'Item_MRP', 'Outlet_Identifier', 'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type', 'Item_Outlet_Sales']


In [ ]:
# Discrete columns
discrete_columns =['Item_Identifier','Item_Fat_Content','Item_Type', 'Outlet_Identifier', 'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']
cont = ['Item_Weight','Item_Visibility','Item_MRP', 'Item_Outlet_Sales']

# Match totals
len(discrete_columns)
len(cont)
len(df.columns)


### Start generating

In [ ]:
# Learn train data
# Takes time
# Some warning messages come. But do not worry
start = time.time()
ctgan = CTGANSynthesizer(epochs=10)
ctgan.fit(df, discrete_columns)
end = time.time()
(end-start)/60

In [16]:
# Get Synthetic data now
datasize = 100000   # Large datasize would require lots of RAM
                    # Be warned, session may crash
                     # So generate in steps and then stack
syn_samples = ctgan.sample(datasize)

In [64]:
# Look at the data
syn_samples.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDC58,12.525694,Low Fat,0.011243,Snack Foods,133.480840,OUT018,1987,Small,Tier 1,Supermarket Type1,590.026066
1,FDW45,12.248238,Regular,0.127689,Meat,121.505663,OUT017,1999,abc,Tier 3,Grocery Store,252.646534
2,FDA50,11.206668,Regular,0.011735,Dairy,206.366793,OUT049,1998,abc,Tier 2,Supermarket Type1,3040.877199
3,NCG18,12.478121,Regular,0.079036,Breakfast,175.567387,OUT045,2007,High,Tier 1,Supermarket Type1,923.593023
4,FDJ09,9.297918,LF,0.070017,Household,154.579841,OUT027,1999,Small,Tier 3,Supermarket Type3,1635.973963


In [18]:
# Save generated data
syn_samples.to_csv("bigdatamart_synthetic.csv",index = False)

In [ ]:
# So file is in gdrive
! pwd
! ls -la '/content/drive/My Drive/Colab_data_files/big_data_mart'

## Set column values back as NULL

In [50]:
# Function to generate null indexes
def generateNaN(df, num_rows_2_setNULL):
  return np.random.choice(df.shape[0], num_rows_2_setNULL, replace=False)

In [53]:
howMany = 10
r = generateNaN(syn_samples, howMany )
r

array([55924, 78145, 54268, 59787, 41776, 32620, 86350, 83465, 96514,
       12232])

In [48]:
# Set column values to NaN
syn_samples.loc[r, 'Outlet_Size'] = np.nan

In [54]:
# Recheck
syn_samples.isnull().sum()

Item_Identifier               0
Item_Weight                   0
Item_Fat_Content              0
Item_Visibility               0
Item_Type                     0
Item_MRP                      0
Outlet_Identifier             0
Outlet_Establishment_Year     0
Outlet_Size                  10
Outlet_Location_Type          0
Outlet_Type                   0
Item_Outlet_Sales             0
dtype: int64

In [55]:
# Save generated data
syn_samples.to_csv("bigdatamart_synthetic.csv",index = False)

In [56]:
# So file is in gdrive
! pwd
! ls -la '/content/drive/My Drive/Colab_data_files/big_data_mart'

/content/drive/My Drive/Colab_data_files/big_data_mart
total 14981
-rw------- 1 root root   869537 Jul  2 09:07 bigdatamart.csv
-rw------- 1 root root 14465731 Jul  2 09:45 bigdatamart_synthetic.csv
drwx------ 2 root root     4096 Jul  2 09:07 .ipynb_checkpoints


In [ ]:
####################### I am done ############